# Chapter 15 - Autoencoders

## PCA with a linear Autoencoder

In [ ]:
import tensorflow as tf

from tensorflow.contrib.layers import fully_connected

# 3d inputs
n_inputs = 3

# Reduce inputs to 2d encodings
n_hidden = 2

# Attempt to recover the initial input
n_outputs = n_inputs

learning_rate = 0.001

# Placeholder for training data; number of examples is unknown (None)
X = tf.placeholder(tf.float32, shape=[None, n_inputs])

# Hidden layer that encodes input data
hidden = fully_connected(X, n_hidden, activation_fn=None)

# Output layer that decodes intermediate representation; note that this
# does not use an activation function, since we want to recover the
# original input data
outputs = fully_connected(hidden, n_outputs, activation_fn=None)

In [ ]:
reconstruction_loss = tf.reduce_mean(tf.square(outputs - X))

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate)

In [ ]:
training_op = optimizer.minimize(reconstruction_loss)

In [ ]:
init = tf.global_variables_initializer()

### Generate training data

In [ ]:
import numpy as np
import numpy.random as rnd

from sklearn.preprocessing import StandardScaler

rnd.seed(4)
m = 200
w1, w2 = 0.1, 0.3
noise = 0.1

angles = rnd.rand(m) * 3 * np.pi / 2 - 0.5
data = np.empty((m, 3))
data[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * rnd.randn(m) / 2
data[:, 1] = np.sin(angles) * 0.7 + noise * rnd.randn(m) / 2
data[:, 2] = data[:, 0] * w1 + data[:, 1] * w2 + noise * rnd.randn(m)

scaler = StandardScaler()
X_train = scaler.fit_transform(data[:100])
X_test = scaler.transform(data[100:])

In [ ]:
n_iterations = 1000
codings = hidden

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        training_op.run(feed_dict={X: X_train})
    codings_val = codings.eval(feed_dict={X: X_test})